In [1]:
import torch
import MiniFL as mfl

In [2]:
def run_client(i:int, client: mfl.algorithms.gd.GDClient):
    client.prepare()
    for _ in range(100):
        _ = client.step()
        
def run_master(master: mfl.algorithms.gd.GDMaster):
    master.prepare()
    for i in range(100):
        print(f"Master: {master.step()}")

In [3]:
from sklearn.datasets import load_svmlight_file

NUM_CLIENTS = 20

data, labels = load_svmlight_file("phishing.txt")
enc_labels = labels.copy()
data_dense = data.todense()


eval_data = (torch.from_numpy(data_dense).to(torch.float32), torch.from_numpy(enc_labels).to(torch.float32)[:, None])
partition_size = (len(eval_data[0]) - 1) // NUM_CLIENTS + 1
clients_data = [(x, y) for x, y in zip(torch.split(eval_data[0], partition_size, dim=0), torch.split(eval_data[1], partition_size, dim=0))]

model = torch.nn.Linear(eval_data[0].shape[1], 1, bias=False)
loss_fn = torch.nn.BCEWithLogitsLoss()

master, clients = mfl.algorithms.get_permk_marina_master_and_clients(
    gamma=100,
    clients_data=clients_data,
    eval_data=eval_data,
    model=model,
    loss_fn=loss_fn,
    p=0.01,
)

In [4]:
import threading

client_threads = []
for i, client in enumerate(clients):
    client_threads.append(threading.Thread(target=run_client, args=(i, client)))
    client_threads[-1].start()
    
master_thread = threading.Thread(target=run_master, args=(master,))
master_thread.start()

master_thread.join()
for t in client_threads:
    t.join()
    


Master: 0.6651992797851562
Master: 0.6752034425735474
Master: 0.6731503009796143
Master: 0.6763127446174622
Master: 0.6752164363861084
Master: 0.6746255159378052
Master: 0.6729355454444885
Master: 0.6732591390609741
Master: 0.6718555092811584
Master: 0.671977698802948
Master: 0.6720216870307922
Master: 0.6720676422119141
Master: 0.6720635890960693
Master: 0.6720393300056458
Master: 0.6720367670059204
Master: 0.6720393300056458
Master: 0.6720397472381592
Master: 0.6720411777496338
Master: 0.6720409393310547
Master: 0.6720410585403442
Master: 0.6720410585403442
Master: 0.6720411777496338
Master: 0.6720407605171204
Master: 0.672040581703186
Master: 0.6720402836799622
Master: 0.6720399856567383
Master: 0.6720404624938965
Master: 0.6720405220985413
Master: 0.6720404624938965
Master: 0.6720404624938965
Master: 0.6720405220985413
Master: 0.6720405220985413
Master: 0.6720405220985413
Master: 0.6720405220985413
Master: 0.6720405220985413
Master: 0.6720405220985413
Master: 0.6720405220985413
Mas

In [5]:
clients[0].data_sender.n_bits_passed

16960